In [3]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

# Loading 2018 LODE Source Data (From R Drive)

In [4]:
path = r'R:/DPOE/LEHD LODES/7.5/Source/OD_Data/Unzipped_RD'

In [5]:
# 2018 source data
csv_2018 = glob.glob(path + "/*2018.csv")
list_2018 = []

for filename in csv_2018:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_2018.append(df)

In [6]:
od_2018 = pd.concat(list_2018, axis=0, ignore_index=True)

In [7]:
import sys
sys.getsizeof(od_2018)

5914258008

# Loading 2021 LODE Source Data (From R Drive)

In [8]:
new_path = r'R:/DPOE/LEHD LODES/7.5_2021_11Nov/Source/LODES_Download_2021-11-19-20-07-45/od'

In [9]:
# new 2018 source data
new_csv_2018 = glob.glob(new_path + "/*2018.csv")
new_list_2018 = []

for filename in new_csv_2018:
    df = pd.read_csv(filename, index_col=None, header=0)
    new_list_2018.append(df)

In [11]:
new_od_2018 = pd.concat(new_list_2018, axis=0, ignore_index=True)

In [12]:
sys.getsizeof(new_od_2018)

5921724168

# Cleaning Old 2018 Source Data

In [13]:
od_2018.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,6.001400e+13,2.500560e+14,1,1,0,0,1,0,0,0,0,1,20201120
1,6.001400e+13,1.324200e+14,1,0,1,0,0,1,0,1,0,0,20201120
2,6.001400e+13,4.803970e+14,1,0,0,1,0,0,1,0,0,1,20201120
3,6.001400e+13,4.104700e+14,1,0,1,0,0,0,1,0,1,0,20201120
4,6.001400e+13,4.505990e+14,1,1,0,0,1,0,0,0,0,1,20201120


In [14]:
od_2018 = od_2018.drop(['createdate',], axis=1)

# Cleaning New 2018 Source Data

In [15]:
new_od_2018.head()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate
0,60014001001007,250056451021050,1,1,0,0,1,0,0,0,0,1,20210907
1,60014001001024,132419702011062,1,0,1,0,0,1,0,1,0,0,20210907
2,60014002001001,480396615012022,1,0,0,1,0,0,1,0,0,1,20210907
3,60014002001016,410470015023004,1,0,1,0,0,0,1,0,1,0,20210907
4,60014002002003,450599210023035,1,1,0,0,1,0,0,0,0,1,20210907


In [16]:
new_od_2018 = new_od_2018.drop(['createdate',], axis=1)

# Filtering for just SD Data (Old Source Data)

In [17]:
string_ints = [str(int) for int in od_2018['w_geocode']]

tf = []
for i in string_ints:
    if i[0:4] == '6073':
        tf.append(True)
    else:
        tf.append(False)
        
f_od_2018 = od_2018[tf]

# Filtering for just SD Data (New Source Data)

In [18]:
string_ints = [str(int) for int in new_od_2018['w_geocode']]

tf = []
for i in string_ints:
    if i[0:4] == '6073':
        tf.append(True)
    else:
        tf.append(False)
        
f_new_od_2018 = new_od_2018[tf]

In [19]:
f_new_od_2018

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
123292,60730001002001,40190046411041,1,0,0,1,0,0,1,0,0,1
123293,60730001002002,40190040732033,1,0,0,1,1,0,0,0,0,1
123294,60730002021006,390450325002034,1,0,1,0,0,1,0,0,0,1
123295,60730002021008,310479685001032,1,0,1,0,0,0,1,0,0,1
123296,60730002023001,390490043004003,1,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
56922040,60730221002002,60730133095006,1,0,1,0,0,0,1,0,0,1
56922041,60730221002002,60730146011009,1,0,1,0,0,0,1,0,0,1
56922042,60730221002002,60730168072006,1,0,1,0,0,0,1,0,0,1
56922043,60730221002002,60730168073009,1,0,1,0,0,0,1,0,0,1


# Comparison 

In [20]:
f_new_od_2018.columns == f_od_2018.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [21]:
f_od_2018.dtypes

w_geocode    float64
h_geocode    float64
S000           int64
SA01           int64
SA02           int64
SA03           int64
SE01           int64
SE02           int64
SE03           int64
SI01           int64
SI02           int64
SI03           int64
dtype: object

In [22]:
f_new_od_2018.dtypes

w_geocode    int64
h_geocode    int64
S000         int64
SA01         int64
SA02         int64
SA03         int64
SE01         int64
SE02         int64
SE03         int64
SI01         int64
SI02         int64
SI03         int64
dtype: object

# Checking the shape of the data

In [23]:
print(f_od_2018.shape)
print(f_new_od_2018.shape)

(4670858, 12)
(4684753, 12)


In [24]:
f_od_2018.shape[0] - f_new_od_2018.shape[0]

-13895

# Checking Length

In [25]:
print(len(f_od_2018))
print(len(f_new_od_2018))

4670858
4684753


# To Numeric and Filter by Geoid

In [27]:
f_od_2018 = f_od_2018.apply(pd.to_numeric)
f_od_2018 = f_od_2018.sort_values(by=['w_geocode'])

In [28]:
f_new_od_2018 = f_new_od_2018.apply(pd.to_numeric)
f_new_od_2018 = f_new_od_2018.sort_values(by=['w_geocode'])

# Count the number of unique IDs

In [30]:
print(f_od_2018['w_geocode'].nunique())
print(f_new_od_2018['w_geocode'].nunique())

17942
17942


# Check if stats are the same

In [31]:
f_od_2018.describe() == f_new_od_2018.describe()

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
count,False,False,False,False,False,False,False,False,False,False,False,False
mean,False,False,False,False,False,False,False,False,False,False,False,False
std,False,False,False,False,False,False,False,False,False,False,False,False
min,True,True,True,True,True,True,True,True,True,True,True,True
25%,True,False,True,True,True,True,True,True,True,True,True,True
50%,False,False,True,True,True,True,True,True,True,True,True,True
75%,True,False,True,True,True,True,True,True,True,True,True,True
max,True,True,True,True,True,True,True,True,True,True,True,True


# Round std to make sure they are identical

In [32]:
# Since this prints out false we see that the rounded version of the standard deviations are all the same
for i in f_od_2018.describe().loc['std',:].round() == f_new_od_2018.describe().loc['std',:].round():
    if i == False:
        print(False)

False
False
